# Remove images with insect in Flowers dataset (test split)

In [156]:
ids = '''
15 
20
40
47
76
78
125
134
179
191
218
231
244
257
258
300
306
331
337
765
1177
1246
1274
1337
1380
1803
1845
1862
1882
1883
1887
1888
1898
1910
1920
1928
1938
1940
1957
2078
2097
2108
2124
2131
2155
2180
2213
2226
2254
2265
2366
2370
2371
2381
2483
2493
2537
2546
2622
2656
2682
2686
2690
2710
2711
2712
2754
2774
2775
2799
2875
2974
2984
2992
2993
3001
3031
3042
3045
3070
3260
3716
3828
3831
3880
3882
3906
3996
4008
4018
4034
4039
4072
4091
4147
4249
4520
4522
4549
4550
4564
4637
4642
4673
4688
4692
4926
4932
4940
4942
4982
5080
5172
5195
5196
5205
5280
5315
5350
5409
5418
5420
5426
5448
5453
5765
5803
5806
5810
5818
5831
5976
6008
6071
6086
6094
6198
6220
6236
6306
6316
6406
6416
6418
6499
6527
6536
6539
6540
6606
6622
6657
6968
6986
6991
7090
7091
7092
7105
7106
7108
7116
7120
7121
7150
7340
7374
7388
7594
7605
7761
7774
7893
7937
7947
7961
7965
8002
8025
8153
4653
6659
3044
7903
1248
5378
4681
6331
6655
6583
4677
7896
6301
8091
6314
3927
2821
6568
4661
7647
6596
7577
6330
6312
2756
7450
41
3865
6302
3037
5391
271
6337
8171
3470
6056
3853
5751
19
'''

In [157]:
insect_ids = sorted(map(int, ids.strip().split()))

In [145]:
import sys
sys.path.append('../')
from tqdm.auto import tqdm
from loader import get_dataloader
from augmentations import get_composed_augmentations
import torch
from tqdm.notebook import tqdm
from torch.nn.functional import softmax
from utils import roc_btw_arr
import numpy as np

In [154]:
flowers_dl = get_dataloader({'dataset': 'Flowers', 
                           'split':'evaluation', 
                           'path': '../datasets', 
                           'batch_size': 64, 
                           'n_workers': 2, 
                           'shuffle':False})

Using No Augmentations
Using No Augmentations


In [158]:
set_test = set(flowers_dl.dataset.indices)
set_insect = set(insect_ids)
set_pure = set_test.difference(set_insect)

In [159]:
len(set_test), len(set_insect), len(set_pure)

(6149, 219, 5930)

In [164]:
pure_ids = np.array(list(set_pure))
np.random.shuffle(pure_ids)
pure_ids

array([  14,  264, 3674, ..., 6488, 7964, 3160], dtype=uint16)

In [165]:
import scipy

In [170]:
np.save('test_wo_insects.npy', pure_ids)

In [168]:
import shutil

In [173]:
for each_ids in pure_ids:
    shutil.copyfile(f'../datasets/flowers/jpg/image_{each_ids:05d}.jpg', f'../datasets/flowers_wo_insects/img_{each_ids:05d}.jpg',)

In [150]:
flowers_dl.dataset.indices

array([6734, 6735, 6737, ..., 8044, 8045, 8047], dtype=uint16)

In [14]:
import matplotlib.pyplot as plt

In [7]:
ds = flowers_dl.dataset

In [33]:
ds.indices

array([6734, 6735, 6737, ..., 8044, 8045, 8047], dtype=uint16)

In [34]:
len(ds.indices)

6149

In [4]:
a = torch.load('../results/RImgNet/vit_hf_md/OOD_rank_Flowers.pkl')

In [5]:
a

array([10122, 10109, 10090, ..., 10056, 10037, 10104])

In [10]:
sorted_idx = np.argsort(a)

In [11]:
sorted_idx

array([2105,  548, 5521, ..., 1902,  925,  825])

In [39]:
vis_img = torch.stack([ds[i][0] for i in sorted_idx[:64]
                      if i not in {sorted_idx[j] for j in [0,1,2,3,4,6,7,10]}])

In [102]:
l_insects = [2105, 548, 5521, 6062, 3856, 798, 2125, 2192, 544, 912, 2122, 6057, 2166, 321, 2175, 2278, 2761, 2232, 2110, 5870,
            917, 5816, 2191, 2174, 2705, 5709, 4136, 610, 2167, 5515, 808, 3606, 2196, 3124, 1080, 487, 602, 462, 4118]

In [116]:
l_insect_indices = ds.indices[l_insects]

In [121]:
torch.save(l_insect_indices, 'remove_flower_index.pkl')

In [114]:
import shutil

In [115]:
! mkdir -p ../datasets/flowers_clean/jpg

In [118]:
len(ds.indices)

6149

In [119]:
len(l_insects)

39

In [120]:
6149 - 39

6110

In [117]:
for idx in ds.indices:
    if idx not in l_insect_indices:
        shutil.copy(f'../datasets/flowers/jpg/image_{idx:05d}.jpg', f'../datasets/flowers_clean/jpg/image_{idx:05d}.jpg')

In [ ]:
[0,1,2,3,4,6,7,10]

In [28]:
from torchvision.utils import make_grid

In [122]:
fgvc_dl = get_dataloader({'dataset': 'FGVC', 
                           'split':'evaluation', 
                           'path': '../datasets', 
                           'batch_size': 64, 
                           'n_workers': 2, 
                           'shuffle':False})

Using No Augmentations
Using No Augmentations


In [123]:
ds = fgvc_dl.dataset

In [126]:
! mkdir ../datasets/FGVC_test_gan

In [127]:
import os

In [128]:
os.path.basename(file)

NameError: name 'file' is not defined

In [131]:
for file, _ in ds.samples:
    shutil.copy(file, '../datasets/FGVC_test_gan/')

In [132]:
a = torch.load('remove_flower_index.pkl')

In [133]:
a


array([4653, 6659, 3044, 7903, 1248, 5378, 4681, 6331, 6655, 6583, 4677,
       7896, 6301, 8091, 6314, 3927, 2821, 6568, 4661, 7647, 6596, 7577,
       6330, 6312, 2756, 7450,   41, 3865, 6302, 3037, 5391,  271, 6337,
       8171, 3470, 6056, 3853, 5751,   19], dtype=uint16)

In [137]:
for aa in a:
    print(aa)

4653
6659
3044
7903
1248
5378
4681
6331
6655
6583
4677
7896
6301
8091
6314
3927
2821
6568
4661
7647
6596
7577
6330
6312
2756
7450
41
3865
6302
3037
5391
271
6337
8171
3470
6056
3853
5751
19


In [136]:
[b.strip().split()]

['15',
 '20',
 '40',
 '47',
 '76',
 '78',
 '125',
 '134',
 '179',
 '191',
 '218',
 '231',
 '244',
 '257',
 '258',
 '300',
 '306',
 '331',
 '337',
 '765',
 '1177',
 '1246',
 '1274',
 '1337',
 '1380',
 '1803',
 '1845',
 '1862',
 '1882',
 '1883',
 '1887',
 '1888',
 '1898',
 '1910',
 '1920',
 '1928',
 '1938',
 '1940',
 '1957',
 '2078',
 '2097',
 '2108',
 '2124',
 '2131',
 '2155',
 '2180',
 '2213',
 '2226',
 '2254',
 '2265',
 '2366',
 '2370',
 '2371',
 '2381',
 '2483',
 '2493',
 '2537',
 '2546',
 '2622',
 '2656',
 '2682',
 '2686',
 '2690',
 '2710',
 '2711',
 '2712',
 '2754',
 '2774',
 '2775',
 '2799',
 '2875',
 '2974',
 '2984',
 '2992',
 '2993',
 '3001',
 '3031',
 '3042',
 '3045',
 '3070',
 '3260',
 '3716',
 '3828',
 '3831',
 '3880',
 '3882',
 '3906',
 '3996',
 '4008',
 '4018',
 '4034',
 '4039',
 '4072',
 '4091',
 '4147',
 '4249',
 '4520',
 '4522',
 '4549',
 '4550',
 '4564',
 '4637',
 '4642',
 '4673',
 '4688',
 '4692',
 '4926',
 '4932',
 '4940',
 '4942',
 '4982',
 '5080',
 '5172',
 '5195',


In [134]:
b = '''
15 
20
40
47
76
78
125
134
179
191
218
231
244
257
258
300
306
331
337
765
1177
1246
1274
1337
1380
1803
1845
1862
1882
1883
1887
1888
1898
1910
1920
1928
1938
1940
1957
2078
2097
2108
2124
2131
2155
2180
2213
2226
2254
2265
2366
2370
2371
2381
2483
2493
2537
2546
2622
2656
2682
2686
2690
2710
2711
2712
2754
2774
2775
2799
2875
2974
2984
2992
2993
3001
3031
3042
3045
3070
3260
3716
3828
3831
3880
3882
3906
3996
4008
4018
4034
4039
4072
4091
4147
4249
4520
4522
4549
4550
4564
4637
4642
4673
4688
4692
4926
4932
4940
4942
4982
5080
5172
5195
5196
5205
5280
5315
5350
5409
5418
5420
5426
5448
5453
5765
5803
5806
5810
5818
5831
5976
6008
6071
6086
6094
6198
6220
6236
6306
6316
6406
6416
6418
6499
6527
6536
6539
6540
6606
6622
6657
6968
6986
6991
7090
7091
7092
7105
7106
7108
7116
7120
7121
7150
7340
7374
7388
7594
7605
7761
7774
7893
7937
7947
7961
7965
8002
8025
8153
'''